<a href="https://colab.research.google.com/github/ztchir/ztchir/blob/main/MECE_694_Office_Building_ENRG_Consump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up notebook to install kaggle datasets
This only need to be run once at the start of the session.

In [ ]:
! pip install kaggle

In [ ]:
# If there is already a directory comment this cell
! mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

Download the Occupant presence and actions in office building Dataset

For more info visit https://www.kaggle.com/claytonmiller/occupant-presence-and-actions-in-office-building



In [ ]:
! kaggle datasets download -d claytonmiller/occupant-presence-and-actions-in-office-building

In [ ]:
! unzip occupant-presence-and-actions-in-office-building.zip -d /content/drive/MyDrive/ColabNotebooks/MECE-694/


In [ ]:
! rm occupant-presence-and-actions-in-office-building.zip

Import Python Packages

In [ ]:
!pip install numpy==1.19.5

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # for visualization

import glob
import re

# Machine Learning Packages
import sklearn as sk
import tensorflow as tf
from tensorflow import keras


from google.colab import drive
drive.mount('/content/drive')

Here we read the data from the separate csv files and combine them in to one data one not it that the data is taken at different time intervals for the some of the data. Outdoor temp and the like are takes on 15 minute intervals. Room, window and ligth data is taken at 15 minute intervals. To fill the missing data points pandas interpolation method is used with linearly interpolates the data on the 15, 30 and 45 minute intervals between the hours.

In [ ]:
#path = r'drive.google.com/drive/my-drive/'
path = r'/content/drive/MyDrive/ColabNotebooks/MECE-694/'
all_files = glob.glob(path + '*.csv')

li = []

for filename in all_files:
  df = pd.read_csv(filename, index_col=None, header=0)
  df.index = pd.to_datetime(df['timestamp [dd/mm/yyyy HH:MM]'])
  df.drop(['timestamp [dd/mm/yyyy HH:MM]'], axis=1, inplace=True)
  df = df.resample('15T').interpolate()
  li.append(df)


In [ ]:
li

In [ ]:
raw_dataset = pd.concat(li, axis = 1)

In [ ]:
raw_dataset.head(10)

Here the columns are renamed for easy python use. The units and descriptions are saved in the units variable.

In [ ]:
raw_dataset.rename(columns={'wind speed [m/s]':'ws [m/s]'}, inplace=True)
raw_dataset.rename(columns={'ki  [1:closed 0:open]':'ki [1:closed 0:open]'}, inplace=True)
feature_info = {}
new_cols = []
plugs = []
windows = []
lights = []
occ = []
temp = []
hum = []
for col in raw_dataset.columns:
  name, unit = col.split(' ', 1)                       
  feature_info[name] = unit
  if unit == '[C]' and name != 'tempOut':
    new_cols.append(name + '_temp')
    temp.append(name + '_temp')
  elif unit == '[%]' and name != 'rh':
    new_cols.append(name + '_hum')
    hum.append(name + '_hum')
  elif unit == '[W]':
    new_cols.append(name + '_plug')
    plugs.append(name + '_plug')
  elif unit == '[1:closed 0:open]':
    new_cols.append(name + '_window')
    windows.append(name + '_window')
  elif unit == '[0:off 1:on]':
    new_cols.append(name + '_light')
    lights.append(name + '_light')
  elif unit == '[0:vacant 1:occupied]':
    new_cols.append(name + '_occ')
    occ.append(name + '_occ')
  else:
    new_cols.append(name)


raw_dataset.columns = new_cols

In [ ]:
continuous = ['tempOut', 'Wind', 'rh', 'gh', 'ws', 'plug_average', 'indoor_temp', 'indoor_humidity']
window_states = ['o1_1_window', 'o1_2_window', 'o1_3_window', 'o1_4_window', 'o2_1_window', 'o2_2_window',
                 'o3_1_window', 'o3_2_window', 'o3_3_window', 'o3_4_window', 'o4_1_window', 'o4_2_window',
                 'mr_1_window', 'mr_2_window', 'mr_3_window', 'mr_4_window', 'mr_5_window', 'mr_6_window']
state = ['occupancy', 'window_state', 'light_state']

In [ ]:
raw_dataset[window_states].hist(figsize=(10,10))

Window state is predominantly closed. And when combined window states are closed at all times. Look at chaning window state per room.

In [ ]:
raw_dataset.head(10)

In [ ]:
occ

In [ ]:
raw_dataset['plug_average'] = raw_dataset[plugs].mean(axis=1)

In [ ]:
raw_dataset['indoor_temp'] = raw_dataset[temp].mean(axis=1)

In [ ]:
raw_dataset['occupancy'] = raw_dataset[occ].any(axis=1).astype(int)

In [ ]:
raw_dataset['window_state'] = raw_dataset[windows].any(axis=1).astype(int)

In [ ]:
raw_dataset['light_state'] = raw_dataset[lights].any(axis=1).astype(int)

In [ ]:
raw_dataset['indoor_humidity'] = raw_dataset[hum].mean(axis=1)

In [ ]:
raw_dataset.head()

In [ ]:
clean_dataset = raw_dataset.drop(occ+windows+lights+temp+hum+plugs, axis=1)

In [ ]:
clean_dataset.to_csv(path + 'clean_data/clean_data.csv', index=True)

In [ ]:
clean_dataset.head()

Notes on dataset.



*   Occupancy on 15 minute time interval Wind Direction on 1 hour. Use linear interpolation between hours to fill missing data. Another option would be to use occupancy in the past hour.
*   Best to match the power load for the building format to maximize usable data.
*  May be useful to use total Equipment power for the building a predicted value.



In [ ]:
# plot the scatter matrix
feature_names = raw_dataset.columns
m = len(raw_dataset.columns) - 1
fig = px.scatter_matrix(raw_dataset, dimensions=feature_names)

fig.update_layout(width=m * 100,
                  height = m * 100,
                  margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [ ]:
raw_dataset.boxplot()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = clean_dataset.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Models
 * Data Analysis/Data Cleaning
 * Evolutionary Feature Selection- Maybe/Exploratory data analysis- Bowen 
 * Linear Regression - Bowen
 ** Non-Linear - Bowen
 * Decision Tree - Bowen
 ** Random Forest - Feature Selection 
 * SVM - Bowen/Zach
 * ANN - Zach
 ** CNN For Time Series Prediction - Zach



In [ ]:
clean_dataset.head(10)

In [ ]:
clean_dataset[continuous].boxplot()

In [ ]:
clean_dataset[state].hist()

Normalize the data.

In [ ]:
normalized_data=(clean_dataset-clean_dataset.mean())/clean_dataset.std()

In [ ]:
clean_dataset

The last few dataset cannot be interpolated and thus need to be removed due to nan values. The window state variable potentially indicates that at least one window is always open. We should revisit these variables. Maybe number of open windows?

In [ ]:
normalized_data = normalized_data.reset_index(drop=True)

In [ ]:
n = 3 # Drop last n columns due to NA values
normalized_data.drop(normalized_data.tail(n).index, inplace=True)

In [ ]:
normalized_data.isna().sum()

  For the following models we are working using time series modeling. Thus the data is not independent. Traditional ramdom data splitting will not work. In this case we will sequentially split the training and test data. We use 80% for tarining and validation and the remaining 20% for testing. 

  

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import TimeSeriesSplit


import matplotlib.pyplot as plt
from matplotlib.patches import Patch


In [ ]:
# Convert dataset into predictors and labels
X = normalized_data.drop(columns = ['window_state', 'plug_average']).to_numpy() # Features to test. Note here we can use past plug data for preditcion
y = normalized_data['plug_average'].to_numpy() # Average Outlet Energy Consumption

# # Perform test train split 80% 20%
# X_train = X[:int(X.shape[0]*0.8)]
# X_test = X[:int(X.shape[0]*0.2)]
# y_train = y[:int(X.shape[0]*0.8)]
# y_test = y[:int(X.shape[0]*0.2)]

In [ ]:
X[0:96].shape[1]

Bowen's Code to get consistent runtime. this is Bowen's code minus his data imports. This code uses the inputs of office occupancy data to predict the energy usage for a 15 minute interval.

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import collections, numpy
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.model_selection import RFECV, rfecv
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

"""3. Exploratory data analysis"""


"""4. Build ML models (linear regression/Random forest/SVM)"""
"""4.1 Using linear regression for prediction"""
reg = LinearRegression()

std = StandardScaler()

REG_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", reg)])

X_train = std.fit_transform(X_train)

# # Grid search to find the proper paramters 
# parameters = {'classifier__C':list(range(0,1000,10)),'classifier__kernel':['linear', 'poly', 'rbf'], 
#               'classifier__gamma':np.array([0.001, 0.01, 0.1, 1])}

# grid_search = GridSearchCV(REG_pipe, param_grid=parameters,scoring='r2',cv=10, n_jobs=-1)

# grid_result = grid_search.fit(X_train, y_train)

# sorted(REG_pipe.get_params().keys())
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Validation using k-fold crossvalidation
start = time.time()
reg = LinearRegression()

std = StandardScaler()

REG_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", reg)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(REG_pipe, X_train, y_train, scoring = "r2", cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")
# =============================================================================
# validation_score['train_score'], validation_score['test_score'] = validation_curve(
#  svc, X_train, y_train, param_name="C", param_range=(1,10,50,60,100,200,300,400,500,600,1000,2000,3000,
#                                                      5000,10000), cv=10)
# =============================================================================
# Testing 
start = time.time()

reg = LinearRegression()

std = StandardScaler()

REG_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", reg)])

REG_pipe.fit(X_train, y_train)

X_test = std.fit_transform(X_test)

y_predict = REG_pipe.predict(X_test)
print('Linear Regression')
print('r2:' , r2_score(y_test,y_predict))
print("Mean absolute error:", mean_absolute_error(y_test, y_predict))
print("Root mean square error:", mean_squared_error(y_test, y_predict, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")


"""4.2 Using SVM for prediction"""
print('Support Vector Machine')
svr = SVR()

std = StandardScaler()

SVR_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", svr)])

X_train = std.fit_transform(X_train)

# # Grid search to find the proper paramters 
# parameters = {'classifier__C':list(range(0,1000,10)),'classifier__kernel':['linear', 'poly', 'rbf'], 
#               'classifier__gamma':np.array([0.001, 0.01, 0.1, 1])}

# grid_search = GridSearchCV(SVR_pipe, param_grid= parameters,scoring='r2',cv=10, n_jobs=-1)

# grid_result = grid_search.fit(X_train, y_train)

# sorted(REG_pipe.get_params().keys())
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Validation using k-fold crossvalidation
start = time.time()
svr = SVR(kernel='rbf')

std = StandardScaler()

SVR_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", svr)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(SVR_pipe, X_train, y_train, scoring = 'r2', cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")
# Testing 
start = time.time()

svr = SVR(C=1.0, epsilon = 0.2)

std = StandardScaler()

SVR_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", svr)])

SVR_pipe.fit(X_train, y_train)

X_test = std.fit_transform(X_test)

y_predict = SVR_pipe.predict(X_test)

print('r2:' , r2_score(y_test,y_predict))
print("Mean absolute error:", mean_absolute_error(y_test, y_predict))
print("Root mean square error:", mean_squared_error(y_test, y_predict, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")

"""4.3 Using RF for prediction"""
print('Random Forest')
rf = RandomForestRegressor(max_depth=2, random_state=0)

std = StandardScaler()

RF_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", rf)])

X_train = std.fit_transform(X_train)

# # Grid search to find the proper paramters 
# parameters = {'classifier__C':list(range(0,1000,10)),'classifier__kernel':['linear', 'poly', 'rbf'], 
#               'classifier__gamma':np.array([0.001, 0.01, 0.1, 1])}

# grid_search = GridSearchCV(RF_pipe, param_grid= parameters,scoring='r2',cv=10, n_jobs=-1)

# grid_result = grid_search.fit(X_train, y_train)

# sorted(REG_pipe.get_params().keys())
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Validation using k-fold crossvalidation
start = time.time()
rf = RandomForestRegressor(max_depth=2, random_state=0)

std = StandardScaler()

RF_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", rf)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(RF_pipe, X_train, y_train, scoring = "r2", cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")

# Testing 
start = time.time()

rf = RandomForestRegressor(max_depth=2, random_state=0)

std = StandardScaler()

RF_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", rf)])

RF_pipe.fit(X_train, y_train)

X_test = std.fit_transform(X_test)

y_predict = RF_pipe.predict(X_test)
print('r2:' , r2_score(y_test,y_predict))
print("Mean absolute error:", mean_absolute_error(y_test, y_predict))
print("Root mean square error:", mean_squared_error(y_test, y_predict, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")

# MLPRegressor

mlpreg = MLPRegressor(random_state=1, max_iter=500)

std = StandardScaler()

MLP_pipe = Pipeline(steps=[('stanadrdscalar', std), ("classifier", mlpreg)])

X_train = std.fit_transform(X_train)

# # Grid search to find the proper paramters 
# parameters = {'classifier__C':list(range(0,1000,10)),'classifier__kernel':['linear', 'poly', 'rbf'], 
#               'classifier__gamma':np.array([0.001, 0.01, 0.1, 1])}

# grid_search = GridSearchCV(MLP_pipe, param_grid= parameters,scoring='r2',cv=10, n_jobs=-1)

# grid_result = grid_search.fit(X_train, y_train)

# sorted(REG_pipe.get_params().keys())
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


# Validation using k-fold crossvalidation
start = time.time()
mlpreg = MLPRegressor(random_state=1, max_iter=500)

std = StandardScaler()

MLP_pipe = Pipeline(steps=[('stanadrdscalar', std), ("classifier", mlpreg)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(MLP_pipe, X_train, y_train, scoring = "r2", cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")


# Testing
start = time.time()

mlpreg = MLPRegressor(random_state=1, max_iter=500)

MLP_pipe = Pipeline(steps=[('stanadrdscalar', std), ("classifier", mlpreg)])

MLP_pipe.fit(X_train, y_train)

X_test = std.fit_transform(X_test)

y_predict = MLP_pipe.predict(X_test)
print('MLP Regressor')
print('r2:' , r2_score(y_test,y_predict))
print("Mean absolute error:", mean_absolute_error(y_test, y_predict))
print("Root mean square error:", mean_squared_error(y_test, y_predict, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")

In the above code we do not consider that the data we have is time dependent. With time series data we are able to use past data in order to help predict the future energy consumption in the data. The test train split and models are differnt. We must create a suitable train and test split for a Time series data set. Here we save the last months worth of data and use it for testing. The remaining data is used for trainng our models. To create our data we use a the past 24 hours of data to predict the next hour of consuption on 15 minute time intervals. We will look at various models using the time series data. MLP Regressor, CNNs and Deep NN with LTSM layers layers.

In [ ]:
# Use TimeSeries Split 
# tscv = TimeSeriesSplit(
#     n_splits=5,
#     max_train_size=10000,
#     test_size=1000,
#     ) # Create test train split that will train on previous 24 hours of opservations and predict the next hour
# print(tscv)

# for train_index, val_index in tscv.split(X_train):
#   X_tr, X_val = X_train[train_index], X_train[val_index]
#   y_tr, y_val = y_train[train_index], y_train[train_index]

# print(X_tr.shape)


In [ ]:
# Convert dataset into predictors and labels
X = normalized_data.drop(columns = ['window_state']).to_numpy() # Features to test. Note here we can use past plug data for preditcion
y = normalized_data['plug_average'].to_numpy() # Average Outlet Energy Consumption

In [ ]:
def generate_samples(X, y):
    input_size = 96 # The past 24 hours of data each of the 96 entries will have n features
    test_size = 2880 # 30 Last 30 Days of the data set. 30 days * 24 Hours * 4 Quarter Hours
    output_size = 4
    try:
      num_features = X.shape[1]
    except:
      num_features = 1

    X_samples = np.empty([len(X)-input_size-test_size-(input_size + output_size), input_size*num_features])
    y_samples = np.empty((len(X)-input_size-test_size-(input_size + output_size), output_size))
    for step_ahead in range(0,len(X)-input_size-test_size-(input_size + output_size)):
        X_samples[step_ahead,:] = X[step_ahead:step_ahead+input_size].flatten()
        y_samples[step_ahead,:] = y[step_ahead+input_size:step_ahead+input_size+output_size]

    
    train_size = int(len(X) - input_size - test_size)

    X_test = np.empty((test_size,input_size*num_features))
    y_test = np.empty((test_size, output_size))
    for step_ahead in range(0,test_size-3):
      X_test[step_ahead,:] =  X[step_ahead+len(X)-test_size-input_size:step_ahead+len(X)-test_size].flatten()
      y_test[step_ahead,:] = y[step_ahead+len(X)-test_size:step_ahead+len(X)-test_size+output_size]

    X_train = X_samples[0:train_size,:]
    y_train = y_samples[0:train_size,:]
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = generate_samples(X,y)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
n_points = 100
X = np.random.randn(100, 10)

percentiles_classes = [0.1, 0.3, 0.6]
y = np.hstack([[ii] * int(100 * perc) for ii, perc in enumerate(percentiles_classes)])

groups = np.hstack([[ii] * 1000 for ii in range(len(X_train))])
groups.shape

In [ ]:

cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 4

# Evenly spaced groups repeated once
groups = np.hstack([[ii] * 10 for ii in range(10)])


def visualize_groups(classes, groups, name):
    # Visualize dataset groups
    fig, ax = plt.subplots()
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )


visualize_groups(y, groups, "no groups")

def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

cvs = [
    TimeSeriesSplit,
]

for cv in cvs:
    this_cv = cv(n_splits=n_splits)
    fig, ax = plt.subplots(figsize=(6, 3))
    plot_cv_indices(this_cv, X, y, groups, ax, n_splits)

    ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )
    # Make the legend fit
    plt.tight_layout()
    fig.subplots_adjust(right=0.7)
plt.show()

In [ ]:
int(X_train.shape[0]/(4))

In [ ]:
# # Use TimeSeries Split 
# tscv = TimeSeriesSplit(
#     n_splits=int(X_train.shape[0]/4),
#     gap=0,
#     max_train_size=4*24, # Consumption to be predicted based on last 24 hour or 96 time steps
#     test_size=4,  # Predict the outlet consumption for the next hour
#     ) # Create test train split that will train on previous 24 hours of opservations and predict the next hour
# print(tscv)

In [ ]:
# for train_index, val_index in tscv.split(X_train):
#   X_tr, X_val = X_train[train_index], X_train[val_index]
#   y_tr, y_val = y_train[train_index], y_train[val_index]


To compare the models we will be using $R^2$ Score. $R^2$ score $R^2 = 1 - \frac{RSS}{TSS}$ = $1 - \frac{\Sigma(y_i - \hat{y_i})^2}{\Sigma(y_i-\bar{y_i})^2}$. In this case a perfect model will have an RSS of 1 and a baseline RSS of 0, a negative value means the model performs worse than the baseline.

Here we train a random forest regressor usnig time series data to predict the next 4 hours of plug consumption.

In [ ]:
# # Use TimeSeries Split 
# tscv = TimeSeriesSplit(
#     n_splits=int(X_train.shape[0]/4),
#     gap=0,
#     max_train_size=4*24, # Consumption to be predicted based on last 24 hour or 96 time steps
#     test_size=4,  # Predict the outlet consumption for the next hour
#     ) # Create test train split that will train on previous 24 hours of opservations and predict the next hour
# print(tscv)

# i = 1
# score = []

# for mf in np.linspace(8, 10, 1):
#   for ne in np.linspace(50, 100, 6):
#     for md in np.linspace(20, 40, 5):
#       for msl in np.linspace(30, 100, 8):
#         rfr = RandomForestRegressor(
#             max_features = int(mf),
#             n_estimators=int(ne),
#             max_depth=int(md),
#             min_samples_leaf = int(msl))
#         rfr.fit(X_train, y_train)
#         score.append([i,
#                       mf,
#                       ne,
#                       md,
#                       msl,
#                       rfr.score(X_test,y_test)])
#   print(score[i])        
#   i += 1
rfr = RandomForestRegressor(random_state=1, max_depth=2).fit(X_train, y_train)
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)


In [ ]:
# Add Cross Validation
# Validation using k-fold crossvalidation
# Validation using k-fold crossvalidation
start = time.time()
rf = RandomForestRegressor(max_depth=2, random_state=0)

std = StandardScaler()

RF_pipe = Pipeline(steps=[('standardscaler', std), ("classifier", rf)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(RF_pipe, X_train, y_train, scoring = "r2", cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")



start = time.time()
rfr = RandomForestRegressor(random_state=1, max_depth=2).fit(X_train, y_train)
#rfr.score(X_test, y_test)
y_pred = rfr.predict(X_test)

print('MLP Regressor Time Series')
print('r2:' , r2_score(y_test, y_pred))
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("Root mean square error:", mean_squared_error(y_test, y_pred, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")


In [ ]:
y_test.shape

In [ ]:
# Validation using k-fold crossvalidation
start = time.time()
mlpreg = MLPRegressor(random_state=1, max_iter=500)

std = StandardScaler()

MLP_pipe = Pipeline(steps=[('stanadrdscalar', std), ("classifier", mlpreg)])

X_train = std.fit_transform(X_train)

validation_score = cross_validate(MLP_pipe, X_train, y_train, scoring = "r2", cv= 10, return_train_score= True)

validation_score['train_score']
Training_score_mean = np.mean(validation_score['train_score'])
print(Training_score_mean)

validation_score['test_score']
validation_score_mean = np.mean(validation_score['test_score'])
print(validation_score_mean)
print(validation_score)

stop = time.time()
print(f"Training + validation time: {stop - start}s")



start = time.time()
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
#regr.score(X_test, y_test)
y_pred = regr.predict(X_test)

print('MLP Regressor Time Series')
print('r2:' , r2_score(y_test,y_pred))
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("Root mean square error:", mean_squared_error(y_test, y_pred, squared=False))

stop = time.time()
print(f"Testing time: {stop - start}s")

Next we use an MLPRegressor for predictions

In [ ]:
# i = 1
# score = []

# for hl in np.linspace(1, 100, 1):
#   for slvr in ['lbfgs', 'sdg', 'adam']:
#     for al in np.linspace(0.0001, 1, 20):
#       for lr in np.linspace(0.0001, 0.01, 10):
#         regr = MLPRegressor(
#             random_state=1,
#             max_iter=10000,
#             hidden_layer_sizes=int(hl),
#             solver=slvr,
#             alpha=al,
#             learning_rate_init=lr,
#             )
#         regr.fit(X_train, y_train)
#         score.append([i,
#                       mf,
#                       ne,
#                       md,
#                       msl,
#                       regr.score(X_test,y_test)])
#     print(score[i])        
#     i += 1

In [ ]:
y_pred_mlp = regr.predict(X_test)
y_pred_rfr = rfr.predict(X_test)

In [ ]:
plt.plot(y_test, label='True Values')
plt.plot(y_pred_mlp, label='MLP Predicted Values')
plt.plot(y_pred_rfr, label='Random Forest Prediction')
plt.legend()
plt.show()

In [ ]:
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

Next look at Deep RNN using LSTM layers

In [ ]:
X = normalized_data['plug_average'].to_numpy() # Average Outlet Energy Consumption # Features to test. Note here we can use past plug data for preditcion
y = normalized_data['plug_average'].to_numpy() # Average Outlet Energy Consumption

In [ ]:
X_train, y_train, X_test, y_test = generate_samples(X,y)

In [ ]:
log_dir = path + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
X_train[0].shape

In [ ]:
deeprnn = keras.models.Sequential([
    keras.layers.SimpleRNN(96, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(500, return_sequences=True),
    keras.layers.Dense(4)                                   
])

In [ ]:
deeprnn.compile(loss="mean_squared_error",
                optimizer="sgd",
                metrics=[tf.keras.metrics.MeanAbsoluteError()]
                )

In [ ]:
history_deeprnn = deeprnn.fit(X_train,y_train, epochs = 20,
                              validation_split=0.3, callbacks=[tensorboard_callback])

In [ ]:
lstm = Sequential([
    keras.layers.LSTM(200, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(200),
    keras.layers.Dense(100)
])

In [ ]:
history_lstm = ltsm.fit(X_train, y_train, epochs = 20,
                        validation_split=0.3, callbacks = [tensorboard_callback])